In [1]:
# 필요한 패키지 불러오기
import pandas as pd
from konlpy.tag import Okt 
okt = Okt()
import gensim
import gensim.corpora as corpora
import os
# os.environ["PYTHONIOENCODING"] = "utf-8”
import pyLDAvis.gensim

In [2]:
# 저장한 데이터 불러오기
df = pd.read_excel('(2)apple_all_airpods.xlsx')
df.head(20)

,ratings,reviews,label
0,5,비싸서 고민되면 사는게 맞다. 1. 배송속도 / 2. 제조년월 / 3. 포장 품질 ...,1
1,4,배송은 엉망이지만 사용 후기는 아주 만족 와.. 대박 방금 받았는데...엄청 안전하...,1
2,5,한번도 안 써본 사람은 있어도 한번만 쓴 사람은 없다. 아이폰6s에서 8로 바꿨더니...,1
3,4,진심 이건 신세계다!!!!!!!!@_@ 그동안 쿠팡에서 로켓배송을 시켜도 한번도 박...,1
4,5,애플의 감성을 담은 블루투스 이어폰 무선 블루투스 이어폰의 새강자 '에어팟'그렇다....,1
5,5,그냥 이어폰이 아닌 생활필수품 사용해보고 후기 남기고 싶어서 좀 지나고 나서야 후기...,1
6,5,가격도 나름 온라인에서 착하고 배송이 짱 빨라서 추천 마침 봄맞이 로켓페이 행사도 ...,1
7,5,역시 애플!!! 솔직히 보기에 선없는 번들 이어폰 끼고 다니는듯해서 망설이다 사용하...,1
8,4,엄청 비쌈; 연결도 잘되고 좋아요!로켓배송이라 여기서 돈 더쥬고 샀는데 괜찮아요정품...,1
9,5,외 않사요? 사세요. 왜 다들 그 큰 돈을 들여가며 이 하얀 콩나물을 샀는지 알 것...,1


In [3]:
negetiveDf = df[df['label'] == 0]
negetiveDf

,ratings,reviews,label
33,2,포장 상태 별로입니다. 귓구멍 작은 분들은 비추 로켓 배송으로 다음날 바로 받아서 ...,0
69,1,등록된 헤드라인이 없습니다 무엇보다 로켓배송으로 빠른배송 너무 좋아요!!!! 줄이없...,0
95,2,등록된 헤드라인이 없습니다 애플 에어팟 구매후기 입니다!지극히 제 개인적인 의견 및...,0
107,1,it is only for Apple 하두 사람들이 평을 좋게 내리길래 아이콘 x ...,0
161,1,등록된 헤드라인이 없습니다 소비자 입장에선 이런거 되게 불쾌합니다..교환한번했는데 ...,0
...,...,...,...
6522,1,서비스센터 방문 초기불량 다시 받는데 5일걸리네요연결안되서 방문했어요,0
6523,1,등록된 헤드라인이 없습니다 불량 중고제품이 도착했습니다. 그리고 그들은 환불을 하지...,0
6524,1,등록된 헤드라인이 없습니다 그래도 가전제품인데 포장이 너무 허술하네요 포장에 좀 신...,0
6525,1,등록된 헤드라인이 없습니다 15일결제18일 수령뭔놈에 로켓배송이 이따구여그리고 안에...,0


In [5]:
# 리뷰 데이터만 가져오기
review = negetiveDf['reviews']

# 빈 리스트를 만들어 명사 토큰을 추출해 저장한다.
review_noun_tk = []
for n in review:
    review_noun_tk.append(okt.nouns(n)) # 명사 형태소만 출력

In [6]:
#한글자로만 추출된 데이터를 제거하기
review_noun_tk2 = []
for d in review_noun_tk:
    item = [i for i in d if len(i) > 1] # 뽑아낸 토큰 중 크기가 1 이상만 추출
    review_noun_tk2.append(item)

## LDA 모델 구축

In [7]:
# 명사 데이터를 사전화 하여 빈도 필터링을 진행한다.
dict = corpora.Dictionary(review_noun_tk2)

In [8]:
# '등록 헤드라인 등록 리뷰 용이'는 리뷰가 없으면 자동생성되는 내용임 
# '버즈'에 대한 크롤링 내용이기에 압도적으로 많을 수 뿐이 없기에 이러한 필요없는 명사 삭제처리
removeWords = ['등록', '헤드라인', '리뷰', '용이', '제품', '에어팟']

ids_to_remove = [dict.token2id[word] for word in removeWords if word in dict.token2id]
dict.filter_tokens(bad_ids=ids_to_remove)

In [9]:
# 딕셔너리화 성공~
print(dict.token2id)

{'가격': 0, '가끔': 1, '가루': 2, '가면': 3, '가지': 4, '겨우': 5, '경우': 6, '고름': 7, '고생': 8, '과정': 9, '구멍': 10, '귀가': 11, '귓구멍': 12, '그냥': 13, '기도': 14, '기장': 15, '꼽자': 16, '다음': 17, '단점': 18, '대부분': 19, '더욱': 20, '덜렁': 21, '때문': 22, '로켓': 23, '리무': 24, '립앤': 25, '마음': 26, '머리': 27, '모양': 28, '바닥': 29, '바로': 30, '박스': 31, '배송': 32, '별로': 33, '부분': 34, '부어': 35, '블루투스': 36, '비올': 37, '뽁뽁': 38, '사서': 39, '사실': 40, '사용': 41, '살짝': 42, '상상': 43, '상태': 44, '소리': 45, '스티커': 46, '아이폰': 47, '압력': 48, '여느': 49, '연골': 50, '연동': 51, '오류': 52, '외로': 53, '외부': 54, '이어팟': 55, '이어폰': 56, '인식': 57, '일단': 58, '자석': 59, '잘못': 60, '장점': 61, '정도': 62, '조절': 63, '차고': 64, '차단': 65, '최근': 66, '충격': 67, '터치': 68, '포장': 69, '하나': 70, '항상': 71, '활동': 72, '후기': 73, '가방': 74, '걱정': 75, '고장': 76, '교환': 77, '구매': 78, '기간': 79, '노래': 80, '다시': 81, '달리': 82, '몇번': 83, '무엇': 84, '배터리': 85, '벌써': 86, '사람': 87, '수정': 88, '시점': 89, '신분': 90, '애플스토어': 91, '어어어어': 92, '어찌': 93, '어찌하여': 94, '얼씬': 95, '연결': 96, '일자': 97, '일주일': 98, '자

In [10]:
# 단어의 빈도수를 확인하기 위해 doc2bow 로 변환한다.
# BoW 모델은 문서의 단어의 빈도수를 표현해준다.
corpus = [dict.doc2bow(word) for word in review_noun_tk2]

In [11]:
# corpus 출력을 통해 (word_id, work_count)의 BoW 구성을 확인
print(corpus)

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 2), (28, 1), (29, 1), (30, 1), (31, 1), (32, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 2), (42, 1), (43, 1), (44, 1), (45, 3), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 3), (62, 2), (63, 2), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1)], [(0, 1), (13, 1), (23, 1), (32, 2), (36, 1), (41, 1), (52, 1), (56, 1), (57, 1), (62, 1), (73, 1), (74, 1), (75, 2), (76, 1), (77, 1), (78, 2), (79, 1), (80, 2), (81, 1), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 2), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 2), (100, 2)

In [12]:
# 토픽의 갯수 k 개 생성
k = 3

#gensim 라이브러리를 사용해 lda 모델을 생성하는 코드 각 옵션은 만들어놓은 
#corpus lda모델
#iterations 각 문서의 주제 할당을 반복하는 횟수
#num_topics 는 위에 지정한 k 값으로 토픽의 갯수
#id2word 각 토큰의 id에 해당하는 토큰을 연결해준다
#passes는 앞서 넣은 corpus를 학습 반복하는 횟수이다 수가 높아지면 더 잘 학습하지만 그만큼 시간이 증가한다.
#workers 쓰레드의 수 (작업자의 수)이다. 보통은 본인의 컴퓨터에 맞춰 설정하게 된다.
lda_model = gensim.models.ldamulticore.LdaMulticore(corpus, iterations = 12, num_topics = k, id2word = dict, passes = 5, workers = 10)

In [13]:
print(lda_model.print_topics(num_topics = k, num_words = 15))

[(0, '0.018*"그냥" + 0.017*"교환" + 0.016*"불량" + 0.015*"소리" + 0.013*"먼지" + 0.011*"구매" + 0.011*"케이스" + 0.011*"이어폰" + 0.010*"상품" + 0.010*"사용" + 0.009*"반품" + 0.009*"애플" + 0.008*"노래" + 0.008*"중고" + 0.007*"프로"'), (1, '0.018*"충전" + 0.018*"사용" + 0.016*"상품" + 0.015*"배송" + 0.015*"박스" + 0.014*"쿠팡" + 0.013*"구매" + 0.012*"교환" + 0.011*"상태" + 0.010*"포장" + 0.010*"진짜" + 0.010*"부분" + 0.008*"세대" + 0.007*"후기" + 0.007*"그냥"'), (2, '0.028*"교환" + 0.014*"소리" + 0.012*"충전" + 0.012*"애플" + 0.011*"연결" + 0.011*"구매" + 0.011*"음질" + 0.010*"처음" + 0.010*"문제" + 0.010*"오른쪽" + 0.009*"그냥" + 0.009*"쿠팡" + 0.008*"배터리" + 0.008*"노래" + 0.008*"왼쪽"')]


In [14]:
# lda모델을 디스플레이로 출력
lda_vis = pyLDAvis.gensim.prepare(lda_model, corpus, dict)
pyLDAvis.display(lda_vis)

### 에어팟 부정 토픽분석을 통해 얻을 수 있는 정보
1. 상품의 부정적인게 아닌 불량품에 대한 리뷰가 대부분임
2. 대부분 무선이어폰에 전체적인 문제점을 주로 말함